In [1]:
!pip install pynvml==11.4.1
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: pynvml
    Found existing installation: pynvml 11.5.0
    Uninstalling pynvml-11.5.0:
      Successfully uninstalled pynvml-11.5.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 907.0 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
time: 701 µs (started: 2023-04-27 09:22:55 +00:00)


In [2]:
from fastkaggle import *
from fastai.vision.all import *
from fasterai.sparse.all import *
from torchvision.models import vgg16_bn

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")


time: 4.21 s (started: 2023-04-27 09:22:55 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 17.8 s (started: 2023-04-27 09:22:59 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ), 
                                   bs=32//2)

time: 6.76 s (started: 2023-04-27 09:23:17 +00:00)


In [5]:
def run_once(func):
    """
    A decorator that runs a function only once and returns the result for all subsequent calls.
    """
    memo = {}
    def wrapper(*args, **kwargs):
        if func not in memo:
            memo[func] = func(*args, **kwargs)
        return memo[func]
    return wrapper


time: 1.52 ms (started: 2023-04-27 09:23:24 +00:00)


In [6]:
@run_once
def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size 
    print('Model size: {} Byte'.format(model_size))


time: 822 µs (started: 2023-04-27 09:23:24 +00:00)


In [7]:
class MemoryCallback(Callback):
    def before_batch(self):
        get_model_file_size(self.learn.model)

time: 6.5 ms (started: 2023-04-27 09:23:24 +00:00)


In [8]:
sp_cb=[
      GradientAccumulation(32), 
      # SparsifyCallback(sparsity=12.5, 
      #                    granularity='filter', 
      #                    context='local', 
      #                    criteria=large_final, 
      #                    schedule=one_cycle
      #                    ),
      MemoryCallback(),
]

time: 3.39 ms (started: 2023-04-27 09:23:24 +00:00)


In [9]:
model = vgg16_bn(num_classes=10)

time: 3.35 s (started: 2023-04-27 09:23:24 +00:00)


In [10]:
learn = Learner(dls, 
                model, 
                metrics=[accuracy,
                         error_rate,
                        ])
learn.unfreeze()
learn.recorder.train_metrics = True

time: 11.4 ms (started: 2023-04-27 09:23:27 +00:00)


In [11]:
learn.fit_one_cycle(1, 3e-5, cbs=sp_cb)


epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.149559,0.206410,0.793590,1.959992,0.317788,0.682212,02:28


Model size: 537239848 Byte
time: 2min 28s (started: 2023-04-27 09:23:27 +00:00)
